In [2]:
! pip install spectral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212 kB 5.2 MB/s 


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
import spectral
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy

In [4]:
print(torch.__version__) 

1.13.0+cu116


In [ ]:
# 对高光谱数据 X 应用 PCA 变换
def applyPCA(X, numComponents):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], numComponents))
    return newX

# 对单个像素周围提取 patch 时，边缘像素就无法取了，因此，给这部分像素进行 padding 操作
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

# 在每个像素周围提取 patch ，然后创建成符合 keras 处理的格式
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    # 给 X 做 padding
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState, stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
# 地物类别
class_num = 10
X = sio.loadmat('/content/drive/MyDrive/AI data/hyperspetral image/WHU_Hi/WHU_Hi_LongKou.mat')['WHU_Hi_LongKou']
y = sio.loadmat('/content/drive/MyDrive/AI data/hyperspetral image/WHU_Hi/WHU_Hi_LongKou_gt.mat')['WHU_Hi_LongKou_gt']
X = X[500:550,:,:]
y = y[500:550,:]

# 用于测试样本的比例
test_ratio = 0.90
# 每个像素周围提取 patch 的尺寸
patch_size = 21
# 使用 PCA 降维，得到主成分的数量
pca_components = 30

print('Hyperspectral data shape: ', X.shape)
print('Label shape: ', y.shape)

print('\n... ... PCA tranformation ... ...')
X_pca = applyPCA(X, numComponents=pca_components)
print('Data shape after PCA: ', X_pca.shape)

print('\n... ... create data cubes ... ...')
X_pca, y = createImageCubes(X_pca, y, windowSize=patch_size)
print('Data cube X shape: ', X_pca.shape)
print('Data cube y shape: ', y.shape)

print('\n... ... create train & test data ... ...')
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X_pca, y, test_ratio)
print('Xtrain shape: ', Xtrain.shape)
print('Xtest  shape: ', Xtest.shape)

# 改变 Xtrain, Ytrain 的形状，以符合 keras 的要求
#Xtrain = Xtrain.reshape(-1, patch_size, patch_size, pca_components, 1)
#Xtest  = Xtest.reshape(-1, patch_size, patch_size, pca_components, 1)
#print('before transpose: Xtrain shape: ', Xtrain.shape) 
#print('before transpose: Xtest  shape: ', Xtest.shape) 

# 为了适应 pytorch 结构，数据要做 transpose
Xtrain = Xtrain.transpose(0, 3, 1, 2)
Xtest  = Xtest.transpose(0, 3, 1, 2)
print('after transpose: Xtrain shape: ', Xtrain.shape) 
print('after transpose: Xtest  shape: ', Xtest.shape) 

Hyperspectral data shape:  (50, 400, 270)
Label shape:  (50, 400)

... ... PCA tranformation ... ...
Data shape after PCA:  (50, 400, 30)

... ... create data cubes ... ...


KeyboardInterrupt: ignored

In [ ]:
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import scipy
scipy.io.savemat('/content/WHU_Hi_LongKou_tr6.mat', mdict={'Xtrain': Xtrain, 'y_cat':ytrain,})
scipy.io.savemat('/content/WHU_Hi_LongKou_te6.mat', mdict={'Xtrain': Xtest, 'y_cat':ytest,})

In [ ]:
X1 = sio.loadmat('/content/WHU_Hi_LongKou_tr1.mat')['Xtrain']
y1 = sio.loadmat('/content/WHU_Hi_LongKou_tr1.mat')['y_cat'].transpose(1,0).squeeze(-1)
X2 = sio.loadmat('/content/WHU_Hi_LongKou_tr2.mat')['Xtrain']
y2 = sio.loadmat('/content/WHU_Hi_LongKou_tr2.mat')['y_cat'].transpose(1,0).squeeze(-1)
X3 = sio.loadmat('/content/WHU_Hi_LongKou_tr3.mat')['Xtrain']
y3 = sio.loadmat('/content/WHU_Hi_LongKou_tr3.mat')['y_cat'].transpose(1,0).squeeze(-1)
X4 = sio.loadmat('/content/WHU_Hi_LongKou_tr4.mat')['Xtrain']
y4 = sio.loadmat('/content/WHU_Hi_LongKou_tr4.mat')['y_cat'].transpose(1,0).squeeze(-1)
X5 = sio.loadmat('/content/WHU_Hi_LongKou_tr5.mat')['Xtrain']
y5 = sio.loadmat('/content/WHU_Hi_LongKou_tr5.mat')['y_cat'].transpose(1,0).squeeze(-1)
X6 = sio.loadmat('/content/WHU_Hi_LongKou_tr6.mat')['Xtrain']
y6 = sio.loadmat('/content/WHU_Hi_LongKou_tr6.mat')['y_cat'].transpose(1,0).squeeze(-1)

In [ ]:
Xtrain = np.concatenate((X1,X2,X3,X4,X5,X6),axis=0)
ytrain = np.concatenate((y1,y2,y3,y4,y5,y6),axis=1)
print("X_cat.shape:",Xtrain.shape)
print("y_cat.shape:",ytrain.shape)

X_cat.shape: (20451, 30, 21, 21)
y_cat.shape: (1, 20451)


In [ ]:
ytrain = ytrain.transpose(1,0).squeeze(-1)

In [ ]:
print("X_cat.shape:",Xtrain.shape)
print("y_cat.shape:",ytrain.shape)

X_cat.shape: (20451, 30, 21, 21)
y_cat.shape: (20451,)


In [ ]:
Xe1 = sio.loadmat('/content/WHU_Hi_LongKou_te1.mat')['Xtrain']
ye1 = sio.loadmat('/content/WHU_Hi_LongKou_te1.mat')['y_cat'].transpose(1,0).squeeze(-1)
Xe2 = sio.loadmat('/content/WHU_Hi_LongKou_te2.mat')['Xtrain']
ye2 = sio.loadmat('/content/WHU_Hi_LongKou_te2.mat')['y_cat'].transpose(1,0).squeeze(-1)
Xe3 = sio.loadmat('/content/WHU_Hi_LongKou_te3.mat')['Xtrain']
ye3 = sio.loadmat('/content/WHU_Hi_LongKou_te3.mat')['y_cat'].transpose(1,0).squeeze(-1)
Xe4 = sio.loadmat('/content/WHU_Hi_LongKou_te4.mat')['Xtrain']
ye4 = sio.loadmat('/content/WHU_Hi_LongKou_te4.mat')['y_cat'].transpose(1,0).squeeze(-1)
Xe5 = sio.loadmat('/content/WHU_Hi_LongKou_te5.mat')['Xtrain']
ye5 = sio.loadmat('/content/WHU_Hi_LongKou_te5.mat')['y_cat'].transpose(1,0).squeeze(-1)
Xe6 = sio.loadmat('/content/WHU_Hi_LongKou_te6.mat')['Xtrain']
ye6 = sio.loadmat('/content/WHU_Hi_LongKou_te6.mat')['y_cat'].transpose(1,0).squeeze(-1)